In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
import lightgbm as ltb
from sklearn.neural_network import MLPClassifier
from copy import deepcopy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# initial some global parameters
static_end = 6246044

# Import data

In [3]:
s010 = pd.read_csv('data/emg.csv', header = None)
label = pd.read_csv('data/grasp.csv', header = None)
grasprepetition = pd.read_csv('data/grasprepetition.csv', header = None)


s010_static = s010.iloc[0:static_end]
label_static = label.iloc[0:static_end]
grasprepetition_static = grasprepetition.iloc[0:static_end]

In [4]:
s010_static.info()
label_static.info()
grasprepetition_static.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6246044 entries, 0 to 6246043
Data columns (total 12 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
 1   1       float64
 2   2       float64
 3   3       float64
 4   4       float64
 5   5       float64
 6   6       float64
 7   7       float64
 8   8       float64
 9   9       float64
 10  10      float64
 11  11      float64
dtypes: float64(12)
memory usage: 571.8 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6246044 entries, 0 to 6246043
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   0       int64
dtypes: int64(1)
memory usage: 47.7 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6246044 entries, 0 to 6246043
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   0       int64
dtypes: int64(1)
memory usage: 47.7 MB


# Plot signals and labels

In [ ]:
index = np.array(s010_static.index)
fig, ax = plt.subplots(1, figsize = (20,10))
line1, = ax.plot(label_static*0.001)
for i in tqdm(range(0,12)):
    ax.plot(index, s010_static[i])
    
line1.set_label('label')
ax.legend()
plt.show()

# Find training set and testing set

In [ ]:
# get the grasprepetition [1,9] as raw training data, [10,12] as raw testing data
train_index = []
test_index = []
for i in tqdm(range(len(grasprepetition_static))):
    if grasprepetition_static.iloc[i,0] in [1,2,3,4,5,6,7,8,9]:
        train_index.append(i)
    if grasprepetition_static.iloc[i,0] in [10,11,12]:
        test_index.append(i)

In [ ]:
trainx = s010_static.iloc[train_index].reset_index(drop=True)
trainy = label_static.iloc[train_index].reset_index(drop=True)

testx1 = s010_static.iloc[test_index].reset_index(drop=True)
testy1 = label_static.iloc[test_index].reset_index(drop=True)


In [ ]:
trainx
trainy

In [ ]:
# a = 379
# trainx.iloc[a-379:a+1,0].mean()
# trainx_aft_window.iloc[a-379,0]

In [ ]:
# add overlapping window to training data
trainx_aft_window = deepcopy(trainx)
trainy_aft_window = deepcopy(trainy)

for i in tqdm(range(379,len(trainx))):
    for j in range(0,12):
        trainx_aft_window.iloc[i,j] = trainx.iloc[i-379:i+1,j].mean()

trainx_aft_window = trainx_aft_window.iloc[379:len(trainx_aft_window),:]
trainy_aft_window = trainy_aft_window.iloc[379:len(trainy_aft_window),:]

In [ ]:
trainx_aft_window = trainx_aft_window.reset_index(drop=True)
trainy_aft_window = trainy_aft_window.reset_index(drop=True)

In [ ]:
trainx_aft_window
trainy_aft_window

In [ ]:
testx1
testy1

In [ ]:
# add overlapping window to testing data
testx1_aft_window = deepcopy(testx1)
testy1_aft_window = deepcopy(testy1)

for i in tqdm(range(379,len(testx1))):
    for j in range(0,12):
        testx1_aft_window.iloc[i,j] = testx1.iloc[i-379:i+1,j].mean()

testx1_aft_window = testx1_aft_window.iloc[379:len(testx1_aft_window),:]
testy1_aft_window = testy1_aft_window.iloc[379:len(testy1_aft_window),:]

### plot training set

In [ ]:
# trainx_aft_window = trainx_aft_window.reset_index(drop = True)
# trainy_aft_window = trainy_aft_window.reset_index(drop= True)

index = np.array(trainx_aft_window.index)
fig, ax = plt.subplots(1, figsize = (20,10))
line1, = ax.plot(trainy_aft_window*0.000001+0.00001)
for i in tqdm(range(0,12)):
    ax.plot(index, trainx_aft_window[i])
    
line1.set_label('label')
ax.legend()
plt.show()

### plot testing set #1

In [ ]:
testx1_aft_window = testx1_aft_window.reset_index(drop = True)
testy1_aft_window = testy1_aft_window.reset_index(drop= True)

index = np.array(testx1_aft_window.index)
fig, ax = plt.subplots(1, figsize = (20,10))
line1, = ax.plot(testy1_aft_window*0.000001+0.00001)
for i in tqdm(range(0,12)):
    ax.plot(index, testx1_aft_window[i])
    
line1.set_label('label')
ax.legend()
plt.show()

### find the testing set #2

In [ ]:
# find the fisrt 200ms which is 385 data point

# find the ten start point
# ten_start_point is for the testing data set #2
counter = 0
label_number = [1,2,3,4,5,6,7,8,9,10]
checked_label = []
ten_start_point = []
for i in label_number:
    for j in tqdm(range(0, len(testy1_aft_window))):
        if testy1_aft_window.iloc[j,0] == i:
            if i not in checked_label:
                counter += 1
                checked_label.append(i)
                ten_start_point.append(j)
        if counter == 10:
            break
            
# find the ten new end point
# ten_end_point is for the testing data set #2
last = 1
ten_end_point = []
label_number = [1,2,3,4,5,6,7,8,9,10]
for j in label_number:
    last = j
    counter = 0
    for i in tqdm(range(ten_start_point[j-1], len(testy1_aft_window))):
        if i == len(testy1_aft_window)-1:
            ten_end_point.append(i)
        current = testy1_aft_window.iloc[i,0]
        if last == current:
            last = current
        else:
            last = current
            counter += 1
        if counter == 1:
            ten_end_point.append(i)
            break
            
testx2_aft_window = testx1_aft_window.iloc[ten_start_point[0]:ten_start_point[0]+385, :]
testx2_aft_window = testx2_aft_window.append(testx1_aft_window.iloc[ten_end_point[0]:ten_end_point[0]+385, :])

for i in range(1,10):
#     if i == 9:
#         testx2_aft_window = testx2_aft_window.append(testx1_aft_window.iloc[ten_start_point[i]:ten_start_point[i]+385, :])
#         testx2_aft_window = testx2_aft_window.append(testx1_aft_window.iloc[ten_end_point[i]:220557, :])
#         break
    testx2_aft_window = testx2_aft_window.append(testx1_aft_window.iloc[ten_start_point[i]:ten_start_point[i]+385, :])
    testx2_aft_window = testx2_aft_window.append(testx1_aft_window.iloc[ten_end_point[i]:ten_end_point[i]+385, :])

testy2_aft_window = testy1_aft_window.iloc[ten_start_point[0]:ten_start_point[0]+385, :]
testy2_aft_window = testy2_aft_window.append(testy1_aft_window.iloc[ten_end_point[0]:ten_end_point[0]+385, :])

for i in range(1,10):
#     if i == 9:
#         testy2_aft_window = testy2_aft_window.append(testy1_aft_window.iloc[ten_start_point[i]:ten_start_point[i]+385, :])
#         testy2_aft_window = testy2_aft_window.append(testy1_aft_window.iloc[ten_end_point[i]:220557, :])
#         break
    testy2_aft_window = testy2_aft_window.append(testy1_aft_window.iloc[ten_start_point[i]:ten_start_point[i]+385, :])
    testy2_aft_window = testy2_aft_window.append(testy1_aft_window.iloc[ten_end_point[i]:ten_end_point[i]+385, :])

### plot testing set #2

In [ ]:
testx2_aft_window = testx2_aft_window.reset_index(drop = True)
testy2_aft_window = testy2_aft_window.reset_index(drop= True)

index = np.array(testx2_aft_window.index)
fig, ax = plt.subplots(1, figsize = (20,10))
line1, = ax.plot(testy2_aft_window*0.000001+0.000001)
for i in tqdm(range(0,12)):
    ax.plot(index, testx2_aft_window[i])
    
line1.set_label('label')
ax.legend()
plt.show()

## SVM model

In [ ]:
# svm_model = SVC()
# svm_model.fit(trainx,trainy)

# expected1 = testy1.values.flatten()
# predicted1 = svm_model.predict(testx1)
# expected2 = testy2.values.flatten()
# predicted2 = svm_model.predict(testx2)

### calculate the accuracy

In [ ]:
# counter = 0
# for i in range(0, len(expected1)):
#     if expected1[i] == predicted1[i]:
#         counter += 1

# accuracy1 = counter/len(expected1)

# counter = 0
# for i in range(0, len(expected2)):
#     if expected2[i] == predicted2[i]:
#         counter += 1

# accuracy2 = counter/len(expected2)

### plot testing results

In [ ]:
# index = np.array(testy1.index)
# fig, ax = plt.subplots(1, figsize = (20,10))
# line1, = ax.plot(predicted1*0.0001+0.001)
# line2, = ax.plot(expected1*0.0001+0.001)
    
# line1.set_label('predicted')
# line2.set_label('expected')
# ax.legend()
# plt.show()

# index = np.array(testy2.index)
# fig, ax = plt.subplots(1, figsize = (20,10))
# line1, = ax.plot(predicted2*0.0001+0.001)
# line2, = ax.plot(expected22*0.0001+0.001)
    
# line1.set_label('predicted')
# line2.set_label('expected')
# ax.legend()
# plt.show()

## KNN model

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors = 5)

In [ ]:
knn_model.fit(trainx_aft_window,trainy_aft_window)

In [ ]:
expected1 = testy1_aft_window.values.flatten()
predicted1 = knn_model.predict(testx1_aft_window)

In [ ]:
expected2 = testy2_aft_window.values.flatten()
predicted2 = knn_model.predict(testx2_aft_window)

### calculate the accuracy

In [ ]:
counter = 0
for i in range(0, len(expected1)):
    if expected1[i] == predicted1[i]:
        counter += 1

accuracy1 = counter/len(expected1)

counter = 0
for i in range(0, len(expected2)):
    if expected2[i] == predicted2[i]:
        counter += 1

accuracy2 = counter/len(expected2)

In [ ]:
accuracy1
accuracy2

### plot testing results

In [ ]:
index = np.array(testy1.index)
fig, ax = plt.subplots(1, figsize = (20,10))
line1, = ax.plot(predicted1*0.0001+0.001)
line2, = ax.plot(expected1*0.0001+0.001)
    
line1.set_label('predicted')
line2.set_label('expected')
ax.legend()
plt.show()

## lightGBM model

In [ ]:
lightGBM_model = ltb.LGBMClassifier()

In [ ]:
lightGBM_model.fit(trainx_aft_window,trainy_aft_window)

In [ ]:
expected1 = testy1_aft_window.values.flatten()
predicted1 = lightGBM_model.predict(testx1_aft_window)

In [ ]:
expected2 = testy2_aft_window.values.flatten()
predicted2 = lightGBM_model.predict(testx2_aft_window)

### calculate the accuracy

In [ ]:
counter = 0
for i in range(0, len(expected1)):
    if expected1[i] == predicted1[i]:
        counter += 1

accuracy1 = counter/len(expected1)

counter = 0
for i in range(0, len(expected2)):
    if expected2[i] == predicted2[i]:
        counter += 1

accuracy2 = counter/len(expected2)

In [ ]:
accuracy1
accuracy2

### plot testing results

In [ ]:
index = np.array(testy1.index)
fig, ax = plt.subplots(1, figsize = (20,10))
line1, = ax.plot(predicted1*0.0001+0.001)
line2, = ax.plot(expected1*0.0001+0.001)
    
line1.set_label('predicted')
line2.set_label('expected')
ax.legend()
plt.show()

## MLP model

In [ ]:
# MLP_model = MLPClassifier(random_state=1, max_iter=300).fit(trainx,trainy)

In [ ]:
# expected1 = testy1.values.flatten()
# predicted1 = MLP_model.predict(testx1)

### calculate the accuracy

In [ ]:
# counter = 0
# for i in range(0, len(expected1)):
#     if expected1[i] == predicted1[i]:
#         counter += 1

# accuracy1 = counter/len(expected1)

In [ ]:
# accuracy1

### plot testing results

In [ ]:
# index = np.array(testy1.index)
# fig, ax = plt.subplots(1, figsize = (20,10))
# line1, = ax.plot(predicted1*0.0001+0.001)
# line2, = ax.plot(expected1*0.0001+0.001)
    
# line1.set_label('predicted')
# line2.set_label('expected')
# ax.legend()
# plt.show()

In [ ]:
# trainx.to_csv('traintestdata/trainx.csv', header=False, index=False)
# trainy.to_csv('traintestdata/trainy.csv', header=False, index=False)

In [ ]:
# testx1.to_csv('traintestdata/testx1.csv', header=False, index=False)
# testy1.to_csv('traintestdata/testy1.csv', header=False, index=False)

In [ ]:
trainx_aft_window.to_csv('traintestdata/mean/trainx_mean.csv', header=False, index=False)
trainy_aft_window.to_csv('traintestdata/mean/trainy_mean.csv', header=False, index=False)
testx1_aft_window.to_csv('traintestdata/mean/testx1_mean.csv', header=False, index=False)
testy1_aft_window.to_csv('traintestdata/mean/testy1_mean.csv', header=False, index=False)
testx2_aft_window.to_csv('traintestdata/mean/testx2_mean.csv', header=False, index=False)
testy2_aft_window.to_csv('traintestdata/mean/testy2_mean.csv', header=False, index=False)